In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal.windows import gaussian
from coloring import colorize
from numpy import exp, dot, mean, fft, cos,sin,cosh,sqrt,absolute, array, transpose, var, conj

import math
from math import pi
import time

from numpy.fft import fft, ifft, fft2, ifft2, fftshift

L = 700
dt = 10
naxis = 500
k0 = 0.01
M0 =1

# Define position vector
xvector = np.linspace(-L,L,naxis,retstep=True)
dx = xvector[1]
xvector = xvector[0]


nt = 196800
wall_thickness = 100
# psi0 = M0*exp(1j*k0*(xvector)*0*1 - xvector*xvector/100000)
psi0 = M0*np.sin(xvector/L/2 * (naxis/2) / (naxis/2 - wall_thickness-1) * pi + pi/2)
psi0[:wall_thickness+1] = 0
psi0[naxis-wall_thickness-1:] = 0
# psi0 = np.roll(psi0, 200)
# psi0 = np.zeros_like(xvector)
# psi0[220:820] = 1
potential = np.linspace(-1.0, 1.0, naxis)
# potential = potential*potential*potential*potential - potential*potential + 1
potential = potential*potential
# potential *= 0.01

changing_potential = np.linspace(-1, 1, naxis)
drive_frequency = 0.037 * 1
potential = np.zeros(naxis) * 0
# potential[4000:4010] = 0.0001
# potential[100:900] = 0

potential[0:wall_thickness] = 100
potential[naxis-wall_thickness:] = 100


klist = np.array([y for y in range(0,naxis//2)] + [0] + [a for a in range(-naxis//2+1,0)])
mu_k = pi * (naxis-1)*klist/(L*naxis)
expmu = exp(-1j*(mu_k**2)*(dt)/2/0.1)
m2 = np.linspace(-1, 1, naxis) ** 2

# for drive_frequency in np.linspace(0.000271*dt, 0.001273*dt, 10):
for drive_frequency in np.linspace(0.000711*dt, 0.001273*dt, 10):
    momentum_sum_list = []
    psit_list = []
    psit = psi0
    psit_list.append(psit)

    drive_list = []
    momentum_list = []
    print(drive_frequency)
    # Computation
    ti = time.time()
    for i in range(1,nt):
        drive_phase = math.sin(i * drive_frequency) * 0.2
        # drive_potential = - 2*drive_phase*changing_potential + drive_phase**2
        drive_potential = changing_potential * drive_phase
        # drive_potential *= 1 if i < 70000 else 0
        psitft = fft(psit)
        psitft = expmu*psitft
        psit = ifft(psitft)
        composite_potential = potential + 1*0.05*drive_potential
        composite_potential *= 0.02
        psit = exp(-1j*composite_potential*dt) * psit

        if i%1000 == 0:
            print(i)

        if i % 5 == 0:
            psit_list.append(psit)
            drive_list.append(composite_potential)
            momentum_list.append(fftshift(psitft))
            momentum_sum_list.append(np.sum(m2*fftshift(psitft)))

    tf = time.time()
    psit_list_abs = map(absolute,psit_list)
    psit_list_phase = map(np.angle,psit_list)
    # print(["Time of computation: ",tf-ti," seconds"])

    # fig = plt.figure(figsize=(9,8))
    ax = plt.subplot(1, 3, 1)
    # gpeplot = plt.imshow((np.absolute(psit_list[:10000])),origin='lower',aspect='auto')
    gpeplot = plt.imshow((np.absolute(psit_list[:100000])), origin='lower', aspect='auto')
    plt.xlabel("Position",fontsize=20)
    plt.ylabel("Time",fontsize=20)
    # plt.colorbar()
    plt.suptitle('Wavefunction amplitude $|\psi|$',fontsize=20)

    # plt.subplot(1, 3, 2)
    # drive_list = np.minimum(0.0001, drive_list)
    # plt.imshow(drive_list[:100000],origin='lower',aspect='auto')

    plt.subplot(1,2,2)
    momentum_list = np.minimum(1, momentum_list)
    # plt.plot(np.convolve(np.absolute(momentum_sum_list), gaussian(5311, 560)))
    # plt.imshow(np.abs(momentum_list), origin='lower', aspect='auto')
    plt.imshow(np.abs(np.fft.fftshift(np.fft.fft(np.array(psit_list)))),origin='lower',aspect='auto')
    # plt.show()
    plt.pause(0.001)

ModuleNotFoundError: No module named 'numpy'